# Triplet Trashbin Classifier 

## Import libraries and main variables

Notebook di presentazione dello studio della realizzazione di una rete Triplet utilizzando `pytorch-lighting`

In [1]:
from libs.Dataset import TripletTrashbinDataModule
from libs.Model import TripletNetwork, TripletNetworkV2, evaluating_performance
from torchvision.models import squeezenet1_1
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import progress
from os.path import join
import warnings # or to ignore all warnings that could be false positives
warnings.filterwarnings("ignore")
from pytorch_lightning.utilities.warnings import PossibleUserWarning
warnings.filterwarnings("ignore", category=PossibleUserWarning)

In [2]:
data_img_size = 224
data_batch_size = 256
# otherwise, return warning
n_workers = 0  # os.cpu_count()
LOGS_FOLDER = "logs"
MAIN_MODELS_FOLDER = "models"
MAX_EPOCHS = 31

dm = TripletTrashbinDataModule(img_size=data_img_size,num_workers=n_workers)
dm.setup()

dm_v2 = TripletTrashbinDataModule(img_size=data_img_size,num_workers=n_workers, trb_train_csv="triplet_training_v2.csv", trb_val_csv="triplet_validation_v2.csv", trb_test_csv="triplet_test_v2.csv")
dm_v2.setup()

dm_v3 = TripletTrashbinDataModule(img_size=data_img_size,num_workers=n_workers, trb_train_csv="triplet_training_v3.csv", trb_val_csv="triplet_validation_v3.csv", trb_test_csv="triplet_test_v3.csv")
dm_v3.setup()



Adatto SqueezeNet, per farlo funzionare come feature extractor, sostituendo il classificatore con il modulo Identity che ritornerà l'input senza alcuna manipolazione.

In [5]:
squeezeNet = squeezenet1_1(pretrained=True)
squeezeNet.classifier = nn.Identity()

squeezeNet(torch.zeros(1,3,data_img_size,data_img_size)).shape

torch.Size([1, 86528])

## Finding best parameters

Imposto il learning rate con quello trovato nella precedente report

In [6]:
tripletNetwork = TripletNetwork(squeezeNet, lr=0.002, batch_size=data_batch_size)

Batch size finder

In [7]:
trainer = pl.Trainer(auto_scale_batch_size="power", max_epochs=-1)
trainer.tune(tripletNetwork, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Batch size 2 succeeded, trying batch size 4
Batch size 4 succeeded, trying batch size 8
Batch size 8 succeeded, trying batch size 16
Batch size 16 succeeded, trying batch size 32
Batch size 32 succeeded, trying batch size 64
Batch size 64 succeeded, trying batch size 128
Batch size 128 succeeded, trying batch size 256
Batch size 256 succeeded, trying batch size 512
Batch size 512 succeeded, trying batch size 1024
Batch size 1024 succeeded, trying batch size 2048
Batch size 2048 succeeded, trying batch size 4096
Batch size 4096 succeeded, trying batch size 8192
Batch size 6600 succeeded, trying batch size 13200
Finished batch size finder, will continue with full run using batch size 6600
Restoring states from the checkpoint path at /Users/danilo/GitHub/deep-learning/.scale_batch_size_cd046314-3d33-42b7-8bb2-55d987f7354e.ckpt


{'scale_batch_size': 6600}

Learning rate finder

In [7]:
trainer = pl.Trainer(auto_lr_find=True)
lr_finder = trainer.tune(tripletNetwork, datamodule=dm)
lr_finder

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

LR finder stopped early after 50 steps due to diverging loss.
Restoring states from the checkpoint path at /Users/danilo/GitHub/triplet-trashbin-classifier/.lr_find_d07c4002-f70a-49de-bfd2-97a2aa460a37.ckpt
Learning rate set to 7.585775750291837e-08


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x139b93940>}

## Models

Loading models from checkpoint and visualize plot. Evaluating classification

### Triplet Margin Loss

In [3]:
ckpt = join(MAIN_MODELS_FOLDER, "tn_TripletMarginLoss_v2-epoch-30.ckpt")

tripletNetwork_tml = TripletNetwork.load_from_checkpoint(checkpoint_path=ckpt) 

# logger_tml = TensorBoardLogger("models/logs", name="tripletNetwork_TripletMarginLoss")

trainer = pl.Trainer(gpus=0,
                    max_epochs=MAX_EPOCHS,
                    callbacks=[progress.TQDMProgressBar()],
#                   logger=logger_tml,
                    accelerator="auto",
                    devices=1 # recommended for validation
                    )

trainer.validate(model=tripletNetwork_tml, ckpt_path=ckpt, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at models/tn_TripletMarginLoss_v2-epoch-30.ckpt
Loaded model weights from checkpoint at models/tn_TripletMarginLoss_v2-epoch-30.ckpt


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       valid/loss            1.383855938911438
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'valid/loss': 1.383855938911438}]

In [3]:
ckpt = join(MAIN_MODELS_FOLDER, "tn_TripletMarginLoss_v2-epoch-30.ckpt")

tripletNetwork_tml = TripletNetwork.load_from_checkpoint(checkpoint_path=ckpt) 

# logger_tml = TensorBoardLogger("models/logs", name="tripletNetwork_TripletMarginLoss")

trainer = pl.Trainer(gpus=0,
                    max_epochs=MAX_EPOCHS,
                    callbacks=[progress.TQDMProgressBar()],
#                   logger=logger_tml,
                    accelerator="auto",
                    devices=1 # recommended for validation
                    )

trainer.validate(model=tripletNetwork_tml, ckpt_path=ckpt, datamodule=dm_v2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at models/tn_TripletMarginLoss_v2-epoch-30.ckpt
Loaded model weights from checkpoint at models/tn_TripletMarginLoss_v2-epoch-30.ckpt


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       valid/loss            1.289524793624878
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'valid/loss': 1.289524793624878}]

In [ ]:
evaluating_performance(tripletNetwork_tml, dm)

### Triplet Margin with Distance Loss

In [5]:
ckpt = join(MAIN_MODELS_FOLDER, "tn_TripletMarginWithDistanceLoss_v2-epoch-30.ckpt")

tripletNetwork_tmwdl = TripletNetworkV2.load_from_checkpoint(checkpoint_path=ckpt) 

# logger_tml = TensorBoardLogger("models/logs", name="tripletNetwork_TripletMarginLoss")

trainer = pl.Trainer(gpus=0,
                    max_epochs=MAX_EPOCHS,
                    callbacks=[progress.TQDMProgressBar()],
#                   logger=logger_tml,
                    accelerator="auto",
                    devices=1 # recommended for validation
                    )

trainer.validate(model=tripletNetwork_tmwdl, ckpt_path=ckpt, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at models/tn_TripletMarginWithDistanceLoss_v2-epoch-30.ckpt
Loaded model weights from checkpoint at models/tn_TripletMarginWithDistanceLoss_v2-epoch-30.ckpt


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       valid/loss           1.3823999166488647
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'valid/loss': 1.3823999166488647}]

In [ ]:
evaluating_performance(tripletNetwork_tml, dm)